In [ ]:
#Import dependants
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import us
from pathlib import Path
from datetime import datetime

(to find the full names of the states for shd_df, I had to import "us". To do this, type "pip install us" on a random cell, once installed, delete and then restart kernel to use)

# Data Cleaning
### Preparing for Analysis

In [ ]:
CSV1Kpath = Path("Resources/Kaggle1DS.csv")
CSV2Kpath = Path("Resources/Kaggle2DS.csv")
CSV3Xpath = Path("Resources/SHD.csv")
CSV1KR = pd.read_csv(CSV1Kpath)
CSV2KR = pd.read_csv(CSV2Kpath)
CSV3XR = pd.read_csv(CSV3Xpath)

In [ ]:
#Data frames
kaggle_1_df = pd.DataFrame(CSV1KR)
kaggle_2_df = pd.DataFrame(CSV2KR)
shd_df = pd.DataFrame(CSV3XR)

### Cleaning Kaggle 1 df

In [ ]:
#kaggle_1_df

In [ ]:
# Organizing columns in desired order #update: took out address
clean_1_df = pd.DataFrame(kaggle_1_df[["school","type","year","date","time","city","state","urbanrural","race","killed","injured","victims","LONCOD","LATCOD"]])

# Convert "Date" column to datetime
clean_1_df["date"] = pd.to_datetime(clean_1_df["date"])

# Format the 'Date' column for display
clean_1_df["date"] = clean_1_df["date"].dt.strftime('%m/%d/%Y')

# Renaming the columns
clean_1_df.rename(columns={"urbanrural": "City Type", "LONCOD": "Longitude","LATCOD": "Latitude"}, inplace=True)

# Proper case the columns
clean_1_df.columns = [col.title() if col.islower() else col for col in clean_1_df]

# Proper case the rows
clean_1_df = clean_1_df.map(lambda x: x.title() if isinstance(x, str) else x)

# Replace NaN with "Unknown"
clean_1_df = clean_1_df.fillna("Unknown")


# Adding the newly labeled data into a new csv for cleanliness
clean_1_path = Path("Outputs/clean_1_final.csv")
clean_1_df.to_csv(clean_1_path, index=False)

clean_1_df.head()

In [ ]:
# Load the dataset as a DataFrame
clean_2_df = pd.DataFrame(kaggle_2_df)

# Added a "Victims" column to match other datasets for easier merging
clean_2_df["Victims"] = clean_2_df["killed"] + clean_2_df["injured"]

# Organizing available columns in desired order
reorganized_clean_2_df = clean_2_df[["school","year","date","time","city","state","killed","injured", "Victims", "long","lat","weapon",
                                       "weapon_source","age_shooter1","gender_shooter1","race_ethnicity_shooter1","shooter_relationship1"]]

# Convert "Date" column to datetime
reorganized_clean_2_df["date"] = pd.to_datetime(reorganized_clean_2_df["date"])

# Format the 'Date' column for display
reorganized_clean_2_df["date"] = reorganized_clean_2_df["date"].dt.strftime("%m/%d/%Y")

# Renaming the columns
reorganized_clean_2_df.rename(columns={"long": "Longitude","lat": "Latitude","weapon_source":"Weapon Source", "age_shooter1":"Age",
                           "race_ethnicity_shooter1":"Race",}, inplace=True)

# Proper case the columns
reorganized_clean_2_df.columns = [col.title() if col.islower() else col for col in reorganized_clean_2_df]

# Proper case the rows
reorganized_clean_2_df = reorganized_clean_2_df.map(lambda x: x.title() if isinstance(x, str) else x)

# Extract school type from the 'School' column and put it in the 'Type' column
reorganized_clean_2_df.loc[reorganized_clean_2_df["School"].str.contains("High School", case=False, na=False), "Type"] = "High School"
reorganized_clean_2_df.loc[reorganized_clean_2_df["School"].str.contains("Middle School", case=False, na=False), "Type"] = "Middle School"
reorganized_clean_2_df.loc[reorganized_clean_2_df["School"].str.contains("Elementary School", case=False, na=False), "Type"] = "Elementary School"
reorganized_clean_2_df.loc[reorganized_clean_2_df["School"].str.contains("Junior High", case=False, na=False), "Type"] = "Middle School"

# Rearranging new columns and the first dataframe into desired order
reorganized_clean_2_df = pd.DataFrame(reorganized_clean_2_df[["School","Type","Year","Date","Time","City","State","Race","Killed","Injured", "Victims",
                                     "Longitude","Latitude",
                                      "Weapon","Weapon Source","Age","Gender_Shooter1","Shooter_Relationship1"]])

# Replace NaN with "Unknown"
reorganized_clean_2_df = reorganized_clean_2_df.fillna("Unknown")

# Adding the newly labeled data into a new csv for cleanliness
clean_2_path = Path("Outputs/clean_2_final.csv")
reorganized_clean_2_df.to_csv(clean_2_path, index=False)

reorganized_clean_2_df.head()


### Cleaning shd_df

In [ ]:
shd_df

In [ ]:
# Organizing available columns in desired order
clean_3_df = pd.DataFrame(shd_df[["school","School_Level","Date","City","State","First_Shot","Bullied","Domestic_Violence","Gang_Related","Preplanned","Quarter","Location"]])

# Convert "Date" column to datetime
clean_3_df["Date"] = pd.to_datetime(clean_3_df["Date"])

# Create the "Year" column by extracting the year from the "Date" column
clean_3_df['Year'] = clean_3_df['Date'].dt.year

# Format the 'Date' column for display
clean_3_df["Date"] = clean_3_df["Date"].dt.strftime('%m/%d/%Y')

# Renaming the columns
clean_3_df.rename(columns={"school":"School", "School_Level": "Type","First_Shot": "Time"}, inplace=True)

# Proper case the columns
clean_3_df.columns = [col.title() if col.islower() else col for col in clean_3_df]

#"weapon", "weapon_source","age_shooter1","gender_shooter1","race_ethnicity_shooter1","shooter_relationship1"

# Extract school type from the 'School' column and put it in the 'Type' column
clean_3_df.loc[clean_3_df["School"].str.contains("High School", case=False, na=False), "Type"] = "High School"
clean_3_df.loc[clean_3_df["School"].str.contains("Middle School", case=False, na=False), "Type"] = "Middle School"
clean_3_df.loc[clean_3_df["School"].str.contains("Elementary School", case=False, na=False), "Type"] = "Elementary School"
clean_3_df.loc[clean_3_df["School"].str.contains("Junior High", case=False, na=False), "Type"] = "Middle School"

# Map state abbreviations to full state names using us package
def get_full_state_name(abbrev):
    try:
        return us.states.lookup(abbrev).name
    except AttributeError:
        return abbrev

clean_3_df["State"] = clean_3_df["State"].apply(get_full_state_name)

# Rearranging new columns and the first dataframe into desired order
clean_3_df = pd.DataFrame(clean_3_df[["School","Type","Date","City","State","Time","Bullied","Domestic_Violence","Gang_Related","Preplanned","Quarter","Location"]])

# Replace NaN with "Unknown"
clean_3_df = clean_3_df.fillna("Unknown")

# Adding the newly labeled data into a new csv for cleanliness
clean_3_path = Path("Outputs/clean_3_final.csv")
clean_3_df.to_csv(clean_3_path, index=False)

clean_3_df.head()

### Merging Data Frames

drop: "Type_x","Year_x","Date_x","Time_x","City_x","State_x"

In [ ]:
# Merging First dfs
merged_df = pd.merge(clean_1_df, reorganized_clean_2_df, on=["School","Race", "Killed", "Injured", "Victims", "Longitude", "Latitude"], how="right") 
clean_merge_df = merged_df.drop(["Type_x","Year_x","Date_x","Time_x","City_x","State_x"],axis = 1)

# Renaming City for final merge
clean_merge_df.rename(columns={"City_y": "City","Type_y": "Type","Year_y":"Year","Date_y":"Date","Time_y":"Time","State_y":"State"}, inplace=True)

clean_merge_df.columns

drop: "City Type", "Victims","Type_x","Date_x","State_y","Time_x"

In [ ]:
# Final Merge
final_df = pd.merge(clean_merge_df, clean_3_df, on=["School","City"])
final_df_merged = final_df.drop(["City Type","Type_x","Date_x","State_y","Time_x"],axis = 1)

# Renaming City for final merge
final_df_merged.rename(columns={"Type_y": "Type","Date_y": "Date","Time_y":"Time","State_x":"State"}, inplace=True)

# Dropping duplicates in our final DataFrame
clean_final_df = final_df_merged.drop_duplicates(subset="Date")

# Adding the newly labeled data into a new CSV for easier access and analysis
final_path = Path("Outputs/Clean_Final_Data.csv")
clean_final_df.to_csv(final_path, index=False)

# Display Final Dataframe
clean_final_df

# Analyzing the Data

In [ ]:
# Annual analysis of # of incidents by year
year_school_df = clean_final_merge.groupby("Year")

#Count of incidents per year
count_year_incident = year_school_df['Year'].count()

count_year_incident

In [ ]:
# Create a bar chart
annual_chart = count_year_incident.plot(kind="bar", figsize=(20,8))

# Rotate x-axis labels
plt.xticks(rotation=45)
plt.xlabel("Incidents")
plt.ylabel("# of Incidents")
plt.title("Incidents per Year")

plt.show()